# Fiber Network

The problem provided in this example is a fiber network with fixed-fixed (both displacement and moments) boundary conditions with a prescribed compressive displacement (i.e. nonhomogenous Dirichlet Boundary Condition) on the top boundary. Each fiber is modeled with 2D geometrically exact beams (i.e. Simo-Reissner Beams). For more information on beams [see here](../force_control/beam/README.md).



<img src="imgs/fiber.png" width="500">


# Install FEniCS and FEniCS arclength in Google Colab environment 
This step will automatically skip if not in a Google Colab environment.

In [1]:
try:
  import google.colab
  !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
  !git clone https://github.com/pprachas/fenics_arclength.git
  %cd fenics_arclength
  !pip install .
  %cd examples/displacement_control

except ImportError:
  pass

In [1]:
%matplotlib inline
from dolfin import *
import numpy as np
import matplotlib.pyplot as plt
from displacement_control_solver import displacement_control # import displacement control formulation of arc-length solver

#Testing
import h5py
import numpy as np

with h5py.File('voronoi.h5', 'r') as f:
    # Read datasets
    nodes = f['/data0'][:]  # Node coordinates (3268, 2)
    elements = f['/data1'][:]  # Element connectivity (3505, 2)
    regions = f['/data2'][:]  # Region labels (3505,)

    # Print summary
    print("Nodes shape:", nodes.shape)
    print("Elements shape:", elements.shape)
    print("Regions shape:", regions.shape)
    print("Node coordinate range:", nodes.min(axis=0), nodes.max(axis=0))
    print("Sample nodes (first 5):", nodes[:5])
    print("Sample elements (first 5):", elements[:5])
    print("Unique region labels:", np.unique(regions))



# Dealing with ufl legacy
try:
    from ufl import diag, Jacobian, shape
except:
    from ufl_legacy import diag, Jacobian, shape


parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["quadrature_degree"] = 3
parameters['reorder_dofs_serial'] = False

ffc_options = {"optimize": True, \
               "eliminate_zeros": True, \
               "precompute_basis_const": True, \
               "precompute_ip_const": True}

Nodes shape: (3268, 2)
Elements shape: (3505, 2)
Regions shape: (3505,)
Node coordinate range: [1.10216952 0.        ] [ 99.19977725 100.        ]
Sample nodes (first 5): [[12.15555307  2.15525833]
 [96.14357372  6.23690175]
 [97.72302373  7.32806088]
 [93.63362689  5.99763266]
 [76.10638116  3.43308003]]
Sample elements (first 5): [[ 25 562]
 [562 563]
 [563 564]
 [564 565]
 [565 566]]
Unique region labels: [1]


/usr4/me700/yegt/.conda/envs/fiber-env/lib/python3.11/site-packages/dolfin/jit/jit.py:121: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  def compile_class(cpp_data, mpi_comm=MPI.comm_world):


## Import Mesh and define function spaces
In the case of 2D beams we also define the rotation matrix about the $z$ axis and directional derivative with respect to the beam centerline.

In [2]:
mesh = Mesh()
with XDMFFile('voronoi.xdmf') as infile:
    infile.read(mesh)
    

Ue = VectorElement("CG", mesh.ufl_cell(), 2, dim=2) # displacement
Te = FiniteElement("CG", mesh.ufl_cell(), 1) # rotation


V = FunctionSpace(mesh, MixedElement([Ue, Te]))   

v_ = TestFunction(V)
u_, theta_ = split(v_)
dv = TrialFunction(V)
v = Function(V, name="Generalized displacement")
u, theta = split(v)

VR = TensorFunctionSpace(mesh, "DG", 0, shape=(2, 2))

V0 = FunctionSpace(mesh, "DG", 0)



Vu = V.sub(0).collapse()
disp = Function(Vu)

Jac = Jacobian(mesh)
gdim = mesh.geometry().dim()
Jac = as_vector([Jac[i, 0] for i in range(gdim)])
g01 = Jac/sqrt(dot(Jac, Jac))
g02 = as_vector([-g01[1],g01[0]])

r01 = outer(g01,as_vector([1,0]))
r02 = outer(g02, as_vector([0,1]))

R0 = r01+r02

#-----------------------Define Functions for beams-----------------------------------#
def tgrad(u): # directional derivative w.r.t. beam centerline
    return dot(grad(u), g01)

def rotation_matrix(theta): # 2D rotation matrix -- there is no need to do rotation parametrization for 2D beams
    return as_tensor([[cos(theta), -sin(theta)],[sin(theta), cos(theta)]])
Rot = rotation_matrix(theta)

/usr4/me700/yegt/.conda/envs/fiber-env/lib/python3.11/site-packages/dolfin/jit/jit.py:46: RuntimeWarning: mpi4py.MPI.Session size changed, may indicate binary incompatibility. Expected 32 from C header, got 40 from PyObject
  if MPI.size(mpi_comm) == 1:


## Define Dirichlet Boundary Conditions


**Note that for the case of displacement control, the FEniCS expression for the applied displacement mujst have be positive to prevent convergence issues.**

For example:

```
apply_disp = Expression("t", t = 0.0, degree = 0)
```
is valid and will not have convergence issues while
```
apply_disp = Expression("-t", t = 0.0, degree = 0)
```
can cause convergence issues.

The direction of applied loading will be determined by the initial load step.

In [3]:
H = 100.0
w = 100.0

def bottom(x, on_boundary):
    return near(x[1], 0, 1e-6) 
def top(x, on_boundary):
    return near(x[1], H,1e-6) 

def left(x, on_boundary):
    return near(x[0], 0,1e-6)
def right(x, on_boundary):
    return near(x[0], w,1e-6)


BC_bot = DirichletBC(V, Constant((0.0,0.0,0.0)), bottom) # fixed displacement and rotation
BC_top_x = DirichletBC(V.sub(0).sub(0), Constant(0.0), top) # fix displacement
BC_top_rot = DirichletBC(V.sub(1), Constant(0.0), top) # fix rotation

apply_disp = Expression("t", t=0.0, degree = 0) # Create expression to compress the top
BC_top_y = DirichletBC(V.sub(0).sub(1),apply_disp,top) # incrementally compress the top 

bcs = [BC_bot, BC_top_y, BC_top_rot, BC_top_x]

## Kinematics and Weak Form

In [4]:
# Kinematics: This is "total" beam formulation
defo = dot(R0.T,dot(Rot.T, g01 + tgrad(u)) - g01)
curv =  tgrad(theta)

In [5]:
# Geometrical properties
S = 1.5*3 # cross-sectional area
I = 3*1.5**3/12 # Area moment
G = 0.0412 # Shear Modulus
nu = 0.5
E = 2*G*(1+nu)

kappa = 5*(1+nu)/(6+5*nu) # Shear correction (Timoshenko)



# Stiffness moduli
ES = E*S
GS = G*kappa*S
EI = E*I

In [6]:
# Constitutive Equations
C_N = diag(as_vector([ES, GS]))

# Applied Load:
F_max = Constant((0.0,0.0))
M_max = Constant(0.0)

elastic_energy = 0.5 * (dot(defo, dot(C_N, defo)) + (EI*curv**2))*dx

F_int = derivative(elastic_energy, v, v_)
F_ext =(-M_max*theta_ + dot(F_max, u_)) * ds
residual = F_int - F_ext
tangent_form = derivative(residual, v, dv)

## Solver
To use our solver we first have to define the type of solver (i.e. displacement control or force control) and solver parameters before using the solver. Note that the correct type of solver has to first be imported (see first cell).
### Solver parameters
Here the parameters for both types of solvers:

>* `psi` : the scalar arc-length parameter. When psi = 1, the method becomes the spherical arc-length method and when psi = 0 the method becomes the cylindrical arc-length method
>* `abs_tol` *(optional)* : absolute residual tolerance for the linear solver (default value: 1e-10)
>* `rel_tol` *(optional)* :  relative residual tolerance for solver; the relative residual is defined as the ration between the current residual and initial residual (default value: DOLFIN_EPS)
>* `lmbda0` : the initial load parameter
>* `max_iter` : maximum number of iterations for the linear solver
>* `solver` *(optional)*: type of linear solver for the FEniCS linear solve function -- default FEniCS linear solver is used if no argument is used.

Aside from these solver parameters, the arguments need to solve the FEA problem must also be passed into the solver:
>* `u` : the solution function
>* `F_int` : First variation of strain energy (internal nodal forces)
>* `F_ext` : Externally applied load (external applied force)
>* `J` : The Jacobian of the residual with respect to the deformation (tangential stiffness matrix)
>* `displacement_factor` : The incremental displacement factor 

The solver can be called by:

`solver = force_control(psi,abs_tol,rel_tol,lmbda0,max_iter,u,F_int,F_ext,bcs,J,displacement_factor,solver)`

### Using the solver
1. Initialize the solver by calling solver.initialize()
2. Iteratively call solver.solve() until desired stopping condition

In [7]:
# Solver Parameters
psi = 1.0
abs_tol = 1.0e-6
lmbda0 = 0.5 # Positive for Stretch
max_iter = 20
solver = 'mumps'

# Set up arc-length solver
solver = displacement_control(psi=psi, lmbda0=lmbda0, max_iter=max_iter, u=v,
                       F_int=F_int, F_ext=F_ext, bcs=bcs, J=tangent_form, displacement_factor = apply_disp, solver = solver)

In [8]:
disp = [v.vector()[:]]
lmbda = [0]
# Function space to compute reaction force at each iteration
v_reac = Function(V)
bcRy = DirichletBC(V.sub(0).sub(1), Constant(1.0), bottom) # take reaction force from the bottom
f_reac = [0.0]
for ii in range(0,55):
    solver.solve()
    if solver.converged:
        # Store whole displacement field
        disp.append(v.vector()[:])
        # Store displacement factor
        lmbda.append(apply_disp.t)
        # Compute and store reaction force
        bcRy.apply(v_reac.vector())
        f_reac.append(assemble(action(residual,v_reac)))

Initializing solver parameters...
Starting initial Displacement Control Control with Newton Method:
Iteration 0: | 
Absolute Residual: 2.7558e+00| Relative Residual: 1.0000e+00
Iteration 1: | 
Absolute Residual: 1.2518e-01| Relative Residual: 4.5425e-02
Iteration 2: | 
Absolute Residual: 1.7504e-02| Relative Residual: 6.3516e-03
Iteration 3: | 
Absolute Residual: 6.4417e-05| Relative Residual: 2.3375e-05
Iteration 4: | 
Absolute Residual: 2.4024e-08| Relative Residual: 8.7175e-09
Iteration 5: | 
Absolute Residual: 1.0697e-14| Relative Residual: 3.8816e-15

Arc-Length Step 1 :
Iteration: 0 
|Total Norm: 1.0697e-14 |Residual: 1.0697e-14 |A: 0.0000e+00| Relative Norm : 1.0000e+00

|Total Norm: 6.4332e-13 |Residual: 6.3320e-13 |A: 1.1369e-13| Relative Norm : 2.7616e-10


## Post Processing
Here we plot the final deformed shape and the equilibrium path.

In [9]:
# Get dof coordinates:
x_dofs = V.sub(0).sub(0).dofmap().dofs()
y_dofs = V.sub(0).sub(1).dofmap().dofs()
theta_dofs = V.sub(1).dofmap().dofs()
dofs = V.tabulate_dof_coordinates()
dof_coords = dofs.reshape((-1, 2))

In [10]:
x_nodal_coord = dof_coords[x_dofs][:,0]
y_nodal_coord = dof_coords[y_dofs][:,1]
# Get nodal values 

# Plot displacement field
disp_x = x_nodal_coord + disp[-1][x_dofs]
disp_y = y_nodal_coord + disp[-1][y_dofs]

plt.figure(figsize=(7,7))
plt.scatter(disp_x, disp_y, marker = '.', c = 'r', label = 'Deformed Configuration')
plt.scatter(x_nodal_coord,y_nodal_coord, marker = '.', c = 'k', alpha = 0.3, label = 'Initial Configuration')

plt.xlabel('x-coordinates')
plt.ylabel('y-coordinates')
plt.axis('equal')
plt.show()

In [11]:
plt.figure(figsize=(7,5))
plt.plot(-np.array(lmbda), f_reac, c='k', marker = 'o')
plt.xlabel('Applied Displacement')
plt.ylabel('Force')
plt.title('Equilibrium path')

Text(0.5, 1.0, 'Equilibrium path')

## Optional: Creating an animation from solution snapshots

In [12]:
from matplotlib import animation, rc

plt.rcParams["animation.html"] = "jshtml"

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111)

ax.set_xlim([0,w])
ax.set_ylim([-10,H+10])

deformed, = ax.plot([], [], lw = 7, c = 'r', label = 'Deformed Configuration', ls = 'None', marker = '.')
init, = ax.plot(x_nodal_coord, y_nodal_coord, c='k', lw = 5, ls = 'None', label = 'Initial Configuration', marker = '.', alpha = 0.3)
ax.legend(loc = 'lower right')

def drawframe(n):
    disp_x = x_nodal_coord + disp[n][x_dofs]
    disp_y = y_nodal_coord + disp[n][y_dofs]
    
    deformed.set_data(disp_x,disp_y)
    return deformed,

plt.close()
# blit=True re-draws only the parts that have changed.
anim = animation.FuncAnimation(fig, drawframe, frames=len(lmbda), interval=40, blit=True)

anim